In [1]:
import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
import os
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

__Python VERSION: 3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
__pyTorch VERSION: 1.2.0
__CUDA VERSION
/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 7301
__Number CUDA Devices: 2
__Devices
Active CUDA Device: GPU 0
Available devices  2
Current cuda device  0
3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
12.1
svmem(total=67477729280, available=39792410624, percent=41.0, used=26440957952, free=35440160768, active=22673412096, inactive=2971639808, buffers=82165760, cached=5514444800, shared=496238592, slab=5049528320)
memory GB: 0.24885940551757812


In [2]:
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

In [3]:
os.chdir('data')

In [4]:
!pwd

/projects/audio-pipelines/data


In [5]:
# hyps = pd.read_csv("sets/full_set/hyps", names=['audio_id'])
# hyps[['audio_id','hyps']] = hyps["audio_id"].str.split(" ", 1, expand=True)

# targets = pd.read_csv("sets/full_set/targets", names=['audio_id'])
# targets[['audio_id','target']] = targets["audio_id"].str.split(" ", 1, expand=True)

# text = pd.read_csv("sets/full_set/text", names=['audio_id'])
# text[['audio_id','text']] = text["audio_id"].str.split(" ", 1, expand=True)

# speakers = pd.read_csv("sets/full_set/utt2spk", names=['audio_id'])
# speakers[['audio_id','speaker']] = speakers["audio_id"].str.split(" ", 1, expand=True)

# images = pd.read_csv("sets/full_set/wav.scp", names=['audio_id'])
# images[['audio_id','audio_path']] = images["audio_id"].str.split(" ", 1, expand=True)

# dataset = pd.merge(hyps, text, how="left")
# dataset = pd.merge(dataset, speakers, how="left")
# dataset = pd.merge(dataset, images, how="left")
# dataset = pd.merge(dataset, targets, how="left")

In [6]:
# dataset['image_path'] = dataset['audio_path'].apply(lambda x : x[:-4] + ".png")
# def remove_absolute(string, prefix='/home/raznem/projects/audio-pipelines/data/'):
#     if string.startswith(prefix):
#         string = string[len(prefix):]
#     return string
    
# dataset['image_path'] = dataset['image_path'].apply(remove_absolute)

In [7]:
# dataset.head()

In [8]:
# dataset.to_csv('data/dataset.csv')

In [9]:
## from allennlp.commands.elmo import ElmoEmbedder
from torchvision import transforms
import numpy as np

try:
    from PIL import Image
except ImportError:
    import Image
    
toImg = transforms.ToPILImage()
toTensor = transforms.ToTensor()
    
class WavImagesLoader(Dataset):
    def __init__(self, csv_path, transform=None):
        self.dataset = pd.read_csv(csv_path)
        self.transform = transform
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        return image, target
    
    def __len__(self):
        return len(self.dataset)
    
    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))    

#     @staticmethod    
#     def flaotTensorToImage(img, mean=0, std=1):
#         """convert a tensor to an image"""
#         img = np.transpose(img.numpy(), (1, 2, 0))
#         img = (img*std+ mean)*255
#         img = img.astype(np.uint8)    
#         return img    
    
    
class ElmoWavImagesLoader(WavImagesLoader):
    def __init__(self, csv_path, text_vecs_path, transform=None):
        super().__init__(csv_path, transform=None)
        self.text_vecs_path = text_vecs_path
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        text_vec = np.zeros((322, 1024))
        tmp = np.load(self.text_vecs_path + self.dataset["audio_id"][key] + ".npy")
        tmp = np.sum(tmp, axis=0)
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp
        
        return image, text_vec, text_len, target
    
    
class ElmoWavVecLoader(ElmoWavImagesLoader):
    def __init__(self, csv_path, text_vecs_path, transform=None):
        super().__init__(csv_path, text_vecs_path, transform=None)
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_vec = self.dataset['image_vec'][key]
        text = self.dataset['hyps'][key]
        text = text.split(' ')
        
        text_vec = np.zeros(322, 1024)
        tmp = np.load(text_vecs_path + self.dataset["audio_id"][key] + ".npy")
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp

        return image_vec, text_vec, text_len, target
    
    
class ElmoLoader(WavImagesLoader):
    def __init__(self, csv_path, text_vec_list, transform=None):
        super().__init__(csv_path, transform=None)
        self.text_vec_list = text_vec_list
            
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        text_vec = np.zeros((322, 1024))
        tmp = self.text_vec_list[key]
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp

        return text_vec, text_len, target

In [10]:
import pickle as pkl
with open("text_vec_list.p", "rb") as f:
    text_vec_list = pkl.load(f)

## Conv net to get vector from Image

In [11]:
dropout = torch.nn.Dropout(p=0.30)
class ConvRes(nn.Module):
    def __init__(self, insize, outsize):
        super(ConvRes, self).__init__()
        drate = .3
        self.math = nn.Sequential(
            nn.BatchNorm2d(insize),
            # nn.Dropout(drate),
            torch.nn.Conv2d(insize, outsize, kernel_size=2, padding=2),
            nn.PReLU(),
        )

    def forward(self, x):
        return self.math(x)


class ConvCNN(nn.Module):
    def __init__(self, insize, outsize, kernel_size=7, padding=2, pool=2, avg=True):
        super(ConvCNN, self).__init__()
        self.avg = avg
        self.math = torch.nn.Sequential(
            torch.nn.Conv2d(insize, outsize, kernel_size=kernel_size, padding=padding),
            torch.nn.BatchNorm2d(outsize),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(pool, pool),
        )
        self.avgpool = torch.nn.AvgPool2d(pool, pool)

    def forward(self, x):
        x = self.math(x)
        if self.avg is True:
            x = self.avgpool(x)
        return x


class Net(nn.Module):
    def __init__(self, output_dim):
        super(Net, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d(1)

        self.cnn1 = ConvCNN(3, 32, kernel_size=7, pool=4, avg=False)
        self.cnn2 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)
        self.cnn3 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)

        self.res1 = ConvRes(32, 64)

        self.features = nn.Sequential(
            self.cnn1, dropout,
            self.cnn2,
            self.cnn3,
            self.res1,
        )

        self.classifier = torch.nn.Sequential(
            nn.Linear(5184, output_dim),
        )
        self.sig = nn.Sigmoid()
  
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
#         x = self.sig(x)
        return x

    
# if use_cuda:
#     lgr.info ("Using the GPU")
#     model = Net(output_dim).cuda() # On GPU
# else:
#     lgr.info ("Using the CPU")
#     model = Net(output_dim) # On CPU

# lgr.info('Model {}'.format(model))

Get maximum length of text

In [12]:
# max_text_length = dataset_dropna['hyps'].apply(lambda x: len(x.split(' '))).max()
# max_text_length

In [13]:
# image_data = ElmoLoader('data/dataset_dropna.csv', text_vec_list)

In [14]:
# image_data[0]

## RNN to encode Elmo sequence

In [15]:
class TextModel(Net):
    def __init__(self, rnn_dim, layers_rnn=1, text_emb_dim=1024):
        super(Net, self).__init__()
        self.layers_rnn = layers_rnn
        self.text_emb_dim = text_emb_dim
        self.rnn_dim = rnn_dim
        self.text_lstm = nn.LSTM(text_emb_dim, self.rnn_dim, num_layers=self.layers_rnn, batch_first=True)
        self.fc_last = nn.Linear(rnn_dim, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, text, text_len):        
#         cnn_out = self.cnn_model(img)
        seq_lengths = text_len
        pack = torch.nn.utils.rnn.pack_padded_sequence(text, seq_lengths, batch_first=True)
        _, (rnn_out, _) = self.text_lstm(pack)
#         print(rnn_out.shape)
        rnn_out = rnn_out.reshape(-1, self.rnn_dim)
#         x = torch.cat([cnn_out, rnn_out], dim=1)
#         print(rnn_out.shape)
        y_pred = self.fc_last(rnn_out)
        y_pred = torch.squeeze(y_pred)
        y_pred = self.sig(y_pred)
        return y_pred

In [16]:
class TextModelDout(Net):
    def __init__(self, rnn_dim, layers_rnn=1, text_emb_dim=1024, dout_rate=0.3):
        super(Net, self).__init__()
        self.layers_rnn = layers_rnn
        self.text_emb_dim = text_emb_dim
        self.rnn_dim = rnn_dim
        self.text_lstm = nn.LSTM(text_emb_dim, self.rnn_dim, num_layers=self.layers_rnn, dropout=dout_rate, batch_first=True)
        self.fc_last = nn.Linear(rnn_dim, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, text, text_len):        
#         cnn_out = self.cnn_model(img)
        seq_lengths = text_len
        pack = torch.nn.utils.rnn.pack_padded_sequence(text, seq_lengths, batch_first=True)
        _, (rnn_out, _) = self.text_lstm(pack)
#         print(rnn_out.shape)
        rnn_out = torch.mean(rnn_out, 0)
        rnn_out = rnn_out.reshape(-1, self.rnn_dim)
#         x = torch.cat([cnn_out, rnn_out], dim=1)
#         print(rnn_out.shape)
        y_pred = self.fc_last(rnn_out)
        y_pred = torch.squeeze(y_pred)
        y_pred = self.sig(y_pred)
        return y_pred

In [17]:
class TextModelDout_v2(Net):
    def __init__(self, rnn_dim, layers_rnn=1, text_emb_dim=1024, dout_rate=0.3):
        super(Net, self).__init__()
        self.layers_rnn = layers_rnn
        self.text_emb_dim = text_emb_dim
        self.rnn_dim = rnn_dim
        self.text_lstm = nn.LSTM(text_emb_dim, self.rnn_dim, num_layers=self.layers_rnn, dropout=dout_rate, batch_first=True)
        self.fc_last = nn.Linear(rnn_dim, 1)
        self.sig = nn.Sigmoid()
        self.dout = nn.modules.Dropout(dout_rate)
        
    def forward(self, text, text_len):        
#         cnn_out = self.cnn_model(img)
        seq_lengths = text_len
        text = self.dout(text)
        pack = torch.nn.utils.rnn.pack_padded_sequence(text, seq_lengths, batch_first=True)
        _, (rnn_out, _) = self.text_lstm(pack)
#         print(rnn_out.shape)
        rnn_out = torch.mean(rnn_out, 0)
        rnn_out = rnn_out.reshape(-1, self.rnn_dim)
#         x = torch.cat([cnn_out, rnn_out], dim=1)
#         print(rnn_out.shape)
        y_pred = self.fc_last(rnn_out)
        y_pred = torch.squeeze(y_pred)
        y_pred = self.sig(y_pred)
        return y_pred

## Validation

In [18]:
# dataset_val = pd.read_csv('data/dataset_val.csv')

In [19]:
# image_data = WavImagesLoader(csv_path)
# dataloader = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [20]:
def rval_acc(result, target):
    result = (result > 0.7).type(torch.int)
    target = target.type(torch.int)
    acc = target == result
    acc = acc.type(torch.float32).sum()/len(acc)
    return acc

def validate(model, loader_val, dtype):
    prediction_list = []
    target_list = []
    with torch.no_grad():
        for i, (text, text_len, y) in enumerate(loader_val):
            text, text_len = text.to(dtype=dtype), text_len.to(dtype=dtype)
            text_len, perm_idx = text_len.sort(0, descending=True)
            text = text[perm_idx]
            y = y[perm_idx]
            pred = model(text, text_len).to("cpu")
            prediction_list.append(pred)
            target_list.append(y)
    result = torch.squeeze(torch.cat(prediction_list))
    target = torch.squeeze(torch.cat(target_list))
    acc = rval_acc(result, target)
    print(f'Acc {acc}')

## Training function

In [21]:
import gc
def textspec_trainer(loader_train, loader_val, model, optimizer, exp_name='', val_every=None, 
                     save_every=None, print_every=None, epochs=1, use_gpu=True, 
                     dtype=torch.float32):
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:1')
    else:
        device = torch.device('cpu')
        
    model.train()
    model.to(device=device)
    model.eval()
    model.to(device='cpu')
    validate(model, loader_val, dtype)
    model.to(device=device)
    model.train()
    for e in range(epochs):
        print('Epoch %d' %e)
        acc = 0
        files_no = 0
        for t, (text, text_len, y) in enumerate(loader_train):
#             img = img.to(device=device, dtype=dtype)
            text = text.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=dtype)
            text_len, perm_idx = text_len.sort(0, descending=True)
            text = text[perm_idx]
#             img = img[perm_idx]
            y = y[perm_idx]
            
            y_pred = model(text, text_len)
            
            loss = loss_fn(y_pred, y)
            
            acc += (torch.round(y_pred.cpu()) == y.cpu()).sum().type(torch.float32)
            files_no += len(text)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            if print_every is not None and t % print_every == 0:
                print('Iteration %d, loss = %.4f; epoch %d' % (t, loss.item(), e))
                print(f'Acc {acc / files_no}')
                
        if val_every is not None and e % val_every == 0:
            model.eval()
            model.to(device='cpu')
            validate(model, loader_val, dtype)
            model.to(device=device)
            model.train()
        if save_every is not None and e % save_every == 0:
            torch.save(model.state_dict(), f'models/{exp_name}_e%d_cnn_rnn.pt' % e)
            gc.collect()
    torch.save(model.state_dict(), f'models/{exp_name}_cnn_rnn.pt')

## Training

In [22]:
import pickle as pkl
with open("text_vec_list.p", "rb") as f:
    text_vec_list = pkl.load(f)

with open("text_vec_list_val.p", "rb") as f:
    text_vec_list_val = pkl.load(f)

In [ ]:
BATCH_SIZE = 128
LEARNING_RATE = 1e-4
USE_GPU = True
TE_DTYPE = torch.float32
CNN_DIM = 32
RNN_DIM = 32
NUM_EPOCHS = 100

loss_fn = torch.nn.BCELoss()
model = TextModelDout_v2(RNN_DIM, layers_rnn=2, dout_rate=0.5)
image_data = ElmoLoader('data/dataset_dropna.csv', text_vec_list)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loader_train = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)
val_data = ElmoLoader('data/dataset_val.csv', text_vec_list_val)
loader_val = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

textspec_trainer(loader_train, loader_val, model, optimizer, exp_name='text_dout_val', 
                 val_every=5, save_every=10, print_every=100, epochs=NUM_EPOCHS,
      use_gpu=USE_GPU, dtype=TE_DTYPE)

Acc 0.5184999704360962
Epoch 0
Iteration 0, loss = 0.6920; epoch 0
Acc 0.5234375
Iteration 100, loss = 0.6783; epoch 0
Acc 0.5713180899620056
Iteration 200, loss = 0.6579; epoch 0
Acc 0.59017413854599
Acc 0.5184999704360962
Epoch 1
Iteration 0, loss = 0.6794; epoch 1
Acc 0.6171875
Iteration 100, loss = 0.6922; epoch 1
Acc 0.6110767126083374
Iteration 200, loss = 0.7014; epoch 1
Acc 0.609880268573761
Epoch 2
Iteration 0, loss = 0.6586; epoch 2
Acc 0.609375
Iteration 100, loss = 0.6482; epoch 2
Acc 0.6098390817642212
Iteration 200, loss = 0.6341; epoch 2
Acc 0.6150886416435242
Epoch 3
Iteration 0, loss = 0.6573; epoch 3
Acc 0.625
Iteration 100, loss = 0.6249; epoch 3
Acc 0.6293317079544067
Iteration 200, loss = 0.6395; epoch 3
Acc 0.6334344148635864
Epoch 4
Iteration 0, loss = 0.6294; epoch 4
Acc 0.625
Iteration 100, loss = 0.6341; epoch 4
Acc 0.6486695408821106
Iteration 200, loss = 0.6037; epoch 4
Acc 0.6537624597549438
Epoch 5
Iteration 0, loss = 0.6205; epoch 5
Acc 0.65625
Iteration 